<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/03.%20Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-retrieval} -->

# Understanding Retrieval Question Answering

### Setup

In [1]:
%pip install -Uqqq rich openai tiktoken wandb langchain unstructured tabulate pdf2image chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, random
from pathlib import Path
import tiktoken
from getpass import getpass
from rich.markdown import Markdown

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [3]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


## Langchain

[LangChain](https://docs.langchain.com/docs/) is a framework for developing applications powered by language models. We will use some of its features in the code below. Let's start by configuring W&B tracing. 

In [4]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "llmapps"

## Parsing documents

We will use a small sample of markdown documents in this notebook. Let's find them and make sure we can stuff them into the prompt. That means they may need to be chunked and not exceed some number of tokens. 

In [5]:
MODEL_NAME = "text-davinci-003"
# MODEL_NAME = "gpt-4"

In [6]:
from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.md")
    return dl.load()

documents = find_md_files('../docs_sample/')
len(documents)

[nltk_data] Downloading package punkt to /home/cmagganas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/cmagganas/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


11

In [7]:
# We will need to count tokens in the documents, and for that we need the tokenizer
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

In [8]:
# function to count the number of tokens in each document
def count_tokens(documents):
    token_counts = [len(tokenizer.encode(document.page_content)) for document in documents]
    return token_counts

count_tokens(documents)

[2047, 2330, 2616, 1199, 665, 2135, 1154, 310, 387, 2592, 763]

We will use `LangChain` built in `MarkdownTextSplitter` to split the documents into sections. Actually splitting `Markdown` without breaking syntax is not that easy. This splitter strips out syntax.
- We can pass the `chunk_size` param and avoid lenghty chunks.
- The `chunk_overlap` param is useful so you don't cut sentences randomly. This is less necessary with `Markdown`

The `MarkdownTextSplitter` also takes care of removing double line breaks and save us some tokens that way.

In [9]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter = MarkdownTextSplitter(chunk_size=1000)
document_sections = md_text_splitter.split_documents(documents)
len(document_sections), max(count_tokens(document_sections))

(90, 438)

let's look at the first section

In [10]:
Markdown(document_sections[0].page_content)

description: >- Collaborate with your colleagues, share results, and track all the experiments across your team    

Teams                                                                                                              

Use W&B Teams as a central workspace for your ML team to build better models faster.                               

Track all the experiments your team has tried so you never duplicate work.                                         

Save and reproduce previously trained models.                                                                      

Share progress and results with your boss and collaborators.                                                       

Catch regressions and immediately get alerted when performance drops.                                              

Benchmark model performance and compare model versions.                                                            

Create a collaborative team                                                                                        

Sign up or log in to your free W&B account.                                                                        

Click Invite Team in the navigation bar.                                                                           

Create your team and invite collaborators.                                                                         

:::info Note: Only the admin of an organization can create a new team. :::                                         

Invite team members

## Embeddings

Let's now use embeddings with a vector database retriever to find relevant documents for a query. 

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)

We can create a retriever from the db now, we can pass the `k` param to get the most relevant sections from the similarity search

In [12]:
retriever = db.as_retriever(search_kwargs=dict(k=3))

In [17]:
query = "What can wandb artifacts do and what are they used for?"
docs = retriever.get_relevant_documents(query)

In [18]:
# Let's see the results
for doc in docs:
    print(doc.metadata["source"])

../docs_sample/dataset-versioning.md
../docs_sample/dataset-versioning.md
../docs_sample/dataset-versioning.md


## Stuff Prompt

We'll now take the content of the retrieved documents, stuff them into prompt template along with the query, and pass into an LLM to obtain the answer. 

In [19]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

context = "\n\n".join([doc.page_content for doc in docs])
prompt = PROMPT.format(context=context, question=query)

Use langchain to call openai chat API with the question

In [20]:
from langchain.llms import OpenAI

llm = OpenAI()
response = llm.predict(prompt)
Markdown(response)

W&B Artifacts help you save and organize machine learning datasets throughout a project's lifecycle. They enable   
you to version data seamlessly, without interrupting your workflow, prepackage data splits, iteratively refine     
datasets, juggle multiple datasets, visualize and share your data workflow, and use core features to upload,       
version, alias, compare, and download data on local or remote file systems, via S3, GCP, or https.

## Using Langchain

Langchain gives us tools to do this efficiently in few lines of code. Let's do the same using `RetrievalQA` chain.

In [21]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
result = qa.run(query)

Markdown(result)

W&B Artifacts are used to save and organize machine learning datasets throughout a project's lifecycle. They help  
to version data seamlessly, without interrupting the workflow, prepackage data splits, iteratively refine datasets,
juggle multiple datasets, visualize and share data workflows, and provide flexible tracking and hosting options.

In [22]:
import wandb
wandb.finish()